In [8]:
import pandas as pd
import dspy
from dspy.evaluate import SemanticF1
import json
import requests
import boto3

In [9]:
# Helper functions to setup datset and evaluation
def create_dataset(path: str) -> list[dspy.Example]:
	dataset = pd.read_excel(path)
	dataset_dict = dataset.to_dict(orient='records')
	dspy_dataset = []

	for row in dataset_dict:
		dspy_dataset.append(dspy.Example(question=row['input'], response=row['expected_output']).with_inputs("question"))

	print(f"Dataset length: {len(dspy_dataset)}")
	return dspy_dataset

def create_sets(dataset: list[dspy.Example], metric: dspy.Module = SemanticF1()):
	trainset, valset, devset, testset = dataset[:15], dataset[15:30], dataset[30:40], dataset[40:]
	evaluate = dspy.Evaluate(devset=devset, metric=metric, num_threads=24, display_progress=True, display_table=3)

	for name, set in zip([trainset, valset, devset, testset], ["trainset", "valset", "devset", "testset"]):
		print(f"{set} length: {len(name)}")

	return trainset, valset, devset, testset, evaluate

In [10]:
dspy_dataset = create_dataset("synthetics/synthetic_dataset_revised.xlsx")
trainset, valset, devset, testset, evaluate = create_sets(dspy_dataset)

Dataset length: 52
trainset length: 15
valset length: 15
devset length: 10
testset length: 12


In [11]:
def search(query: str, top_k: int) -> list[str]:
    url = "http://localhost:8000/api/llm/retrieval"
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json"
    }
    data = {
        "query": query,
        "top_k": top_k
    }

    documents = requests.post(url, headers=headers, json=data).json()["documents"]
    return [f"[{i}]" + doc["doc_title"] + doc["url"] + "\n\n" + doc["content"] for i, doc in enumerate(documents)]

class TitanLM(dspy.LM):
    def __init__(self, model: str, client, max_tokens: int = 1024, temperature: float = 0.7, top_p: float = 0.9, **kwargs):
        self.client = client
        self.history = []
        self.temperature = temperature
        self.max_tokens = 1024
        self.top_p = top_p

        super().__init__(model, **kwargs)
        self.model = model
    
    def _format_message(self, prompt: str):
        body = json.dumps(
            {
                "inputText": prompt,
                "textGenerationConfig": {
                    "maxTokenCount": self.max_tokens,
                    "stopSequences": [],
                    "temperature": self.temperature,
                    "topP": self.top_p,
                },
            }
        )
        return body

    def generate_content(self, prompt: str) -> str:
        body = self._format_message(prompt)
        response = self.client.invoke_model(
            body=body,
            modelId=self.model,
            accept="application/json",
            contentType="application/json",
        )
        response_body = json.loads(response.get("body").read())
        return response_body.get("results")

    def __call__(self, prompt=None, messages=None, **kwargs):
        # Custom chat model working for text completion model
        prompt = '\n\n'.join([x['content'] for x in messages] + ['BEGIN RESPONSE:'])

        completions = self.generate_content(prompt)
        self.history.append({"prompt": prompt, "completions": completions})

        # Must return a list of strings
        return [completions[0].get("outputText")]

    def inspect_history(self):
        for interaction in self.history:
            print(f"Prompt: {interaction['prompt']} -> Completions: {interaction['completions']}")

In [12]:
class GenerateCitedParagraph(dspy.Signature):
    """Generate a paragraph with citations."""
    context = dspy.InputField(desc="contain relevant information to help answer question.")
    question = dspy.InputField()
    response = dspy.OutputField(
        desc="""
        Includes citations with links to source. 
        Return sources in the way they are presented in the context (with format [x] and links).
        """
    )

class CitedRAG(dspy.Module):
    def __init__(self, num_docs=30):
        self.num_docs = num_docs
        self.respond = dspy.ChainOfThought(GenerateCitedParagraph)

    def forward(self, question):
        context = search(question, top_k=self.num_docs)
        return self.respond(context=context, question=question)

In [13]:
lm = TitanLM("amazon.titan-text-premier-v1:0", client=boto3.client("bedrock-runtime"))
dspy.configure(lm=lm)

In [14]:
rag = CitedRAG()
rag(question="How do I increase my data center efficiency?")

Prediction(
    reasoning='The user is asking for a way to increase data center efficiency. One way to increase data center efficiency is by using an Uninterruptible Power Supply (UPS). UPS can provide high efficiency up to 99% and a few UPS manufacturers can even achieve output power quality of Class 1 as defined by IEC62040-3.',
    response='You can increase your data center efficiency by using an Uninterruptible Power Supply (UPS). UPS can provide high efficiency up to 99% and a few UPS manufacturers can even achieve output power quality of Class 1 as defined by IEC62040-3.'
)

In [15]:
evaluate(CitedRAG())

Average Metric: 2.2277331364287885 / 10  (22.3): 100%|██████████| 10/10 [00:45<00:00,  4.54s/it]
2024/11/09 11:55:50 INFO dspy.evaluate.evaluate: Average Metric: 2.2277331364287885 / 10 (22.3%)


,question,example_response,reasoning,pred_response,SemanticF1
0,"What factors, including multi-tenancy and operator incentives, contribute to cloud's lower environmental impact?","Cloud services generally have a lower environmental impact due to several key factors: 1. **Multi-tenancy**: This concept allows multiple customers to share resources, leading to...",The relevant information to answer the above question is not available in the given context.,"The factors contributing to cloud's lower environmental impact, including multi-tenancy and operator incentives, are not available in the given context.",
1,"Identify how virtualization, power management, and cloud computing reduce energy waste and enhance business operations in server rooms.","Virtualization reduces energy waste in server rooms by allowing multiple virtual machines to run on a single physical server, optimizing hardware usage and decreasing the...","To answer the question, consider the following: Virtualization: Virtualization involves running multiple virtual machines on a single physical server, allowing for better resource utilization and...","Virtualization, power management, and cloud computing can help reduce energy waste and enhance business operations in server rooms by enabling better resource utilization, optimizing energy...",✔️ [0.538]
2,How do virtualization and cloud computing synergistically enhance server room energy efficiency and utilization rates?,Virtualization and cloud computing work together to significantly enhance server room energy efficiency and utilization rates by improving resource allocation and reducing energy consumption. Virtualization...,There is no information regarding virtualization and cloud computing synergistically enhance server room energy efficiency and utilization rates.,There is no information regarding virtualization and cloud computing synergistically enhance server room energy efficiency and utilization rates.,


22.28

In [21]:
from pprint import pprint
pprint(lm.history[-1].get('prompt'))

('Your input fields are:\n'
 '1. `question` (str)\n'
 '2. `ground_truth` (str)\n'
 '3. `system_response` (str)\n'
 '\n'
 'Your output fields are:\n'
 '1. `reasoning` (str)\n'
 '2. `recall` (float): fraction (out of 1.0) of ground truth covered by the '
 'system response\n'
 '3. `precision` (float): fraction (out of 1.0) of system response covered by '
 'the ground truth\n'
 '\n'
 'All interactions will be structured in the following way, with the '
 'appropriate values filled in.\n'
 '\n'
 '[[ ## question ## ]]\n'
 '{question}\n'
 '\n'
 '[[ ## ground_truth ## ]]\n'
 '{ground_truth}\n'
 '\n'
 '[[ ## system_response ## ]]\n'
 '{system_response}\n'
 '\n'
 '[[ ## reasoning ## ]]\n'
 '{reasoning}\n'
 '\n'
 '[[ ## recall ## ]]\n'
 '{recall}        # note: the value you produce must be a single float value\n'
 '\n'
 '[[ ## precision ## ]]\n'
 '{precision}        # note: the value you produce must be a single float '
 'value\n'
 '\n'
 '[[ ## completed ## ]]\n'
 '\n'
 'In adhering to this struc